# **Homework 2-1 Phoneme Classification**

* Slides: https://speech.ee.ntu.edu.tw/~hylee/ml/ml2021-course-data/hw/HW02/HW02.pdf
* Video (Chinese): https://youtu.be/PdjXnQbu2zo
* Video (English): https://youtu.be/ESRr-VCykBs


## The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task,
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [ ]:
import numpy as np
from pathlib import Path

print("Loading data...")
data_root = "../Data/timit_11"
data_root = Path(data_root)

train = np.load(data_root / "train_11.npy")
test = np.load(data_root / "test_11.npy")
train_label = np.load(data_root / "train_label_11.npy")


print(f"Size of training data: {train.shape}")
print(f"Size of testing data: {test.shape}")
print(f"Size of training label: {train_label.shape}")

# Create Dataset

In [ ]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, data, label = None):
        self.data = data
        if label is not None:
            label = label.astype(np.int32)
            self.label = torch.LongTensor(label)
        else:
            self.label = None
            
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [ ]:
VAL_RATIO = 0.2

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y = train[:percent], train_label[:percent]
val_x, val_y = train[percent:], train_label[percent:]

print("Size of training set:", train_x.shape)
print("Size of validation set:", val_x.shape)

Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [ ]:
BATCH_SIZE = 64

from torch.utils.data import DataLoader

train_dataset = TIMITDataset(train_x, train_y)
val_dataset = TIMITDataset(val_x, val_y)
test_dataset = TIMITDataset(test)

#only shuffle the training data
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [ ]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

## Create Model
Define model architecture, you are encouraged to change and experiment with the model architecture.

In [2]:
import torch
import torch.nn as nn


class PhonemeClassifier(nn.Module):
    def __init__(self):
        super(PhonemeClassifier, self).__init__()
        self.layer1 = nn.Linear(429, 1024)
        self.layer2 = nn.Linear(1024, 512)
        self.layer3 = nn.Linear(512, 128)
        self.out = nn.Linear(128, 39)
        
        self.act_fn = nn.Sigmoid()
        
    def forward(self, x):
        x = self.act_fn(self.layer1(x))
        x = self.act_fn(self.layer2(x))
        x = self.act_fn(self.layer3(x))
        x = self.out(x)
        return x
    
model = PhonemeClassifier()
print(model)

PhonemeClassifier(
  (layer1): Linear(in_features=429, out_features=1024, bias=True)
  (layer2): Linear(in_features=1024, out_features=512, bias=True)
  (layer3): Linear(in_features=512, out_features=128, bias=True)
  (out): Linear(in_features=128, out_features=39, bias=True)
  (act_fn): Sigmoid()
)


In [ ]:
class PhonemeClassifier1(nn.Module):
    def __init__(self):
        super(PhonemeClassifier1, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(429, 2048),
            nn.BatchNorm1d(2048),
            nn.Sigmoid(),
            nn.Dropout(0.25),
            
            nn.Linear(2048, 1024),
            nn.BatchNorm1d(1024),
            nn.Sigmoid(),
            nn.Dropout(0.25),
            
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.Sigmoid(),
            nn.Dropout(0.25),
            
            
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.Sigmoid(),
            
            # output layer
            nn.Linear(128, 39)
        )
        
    def forward(self, x):
        return self.model(x)

model1 = PhonemeClassifier1()
print(model1)

PhonemeClassifier1(
  (model): Sequential(
    (0): Linear(in_features=429, out_features=2048, bias=True)
    (1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Sigmoid()
    (3): Dropout(p=0.25, inplace=False)
    (4): Linear(in_features=2048, out_features=1024, bias=True)
    (5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Sigmoid()
    (7): Dropout(p=0.25, inplace=False)
    (8): Linear(in_features=1024, out_features=512, bias=True)
    (9): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): Sigmoid()
    (11): Dropout(p=0.25, inplace=False)
    (12): Linear(in_features=512, out_features=128, bias=True)
    (13): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): Sigmoid()
    (15): Linear(in_features=128, out_features=39, bias=True)
  )
)


# Training

In [ ]:
# Fix random seeds for reproducibility.
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    
    
# fix random seed for reproducibility
same_seeds(0)

# get device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'DEVICE: {device}')

# training parameters
num_epoch = 20               # number of training epoch
learning_rate = 0.0001       # learning rate

# the path where checkpoint saved
model_path = './model.ckpt'

# create model, define a loss function, and optimizer
model = PhonemeClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# start training

best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    
    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad() 
        outputs = model(inputs) 
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 
        
        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()
        
        
    if len(val_dataset) > 0:
        model.eval()
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels)
                _, pred = torch.max(outputs, 1)
                val_acc += (pred.cpu() == labels.cpu()).sum().item()
                val_loss += batch_loss.item()
                
            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_dataset), train_loss/len(train_loader), val_acc/len(val_dataset), val_loss/len(val_loader)
            ))
            
            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_dataset)))
                
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_dataset), train_loss/len(train_loader)
        ))
        
# if not validating, save the last epoch
if len(val_dataset) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')
    

# Testing

In [ ]:
model = PhonemeClassifier().to(device)
model.load_state_dict(torch.load(model_path))

predictions = []
model.eval()

with torch.no_grad():
    for i, data in enumerate(test_dataset):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, pred = torch.max(outputs, 0)
        
        predictions += pred.cpu().numpy().tolist()
        
with open("prediction.csv", 'w') as f:
    f.write("Id,Category\n")
    for i, pred in enumerate(predictions):
        f.write(f"{i},{pred}\n")